In [2]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from brainiak.fcma.util import compute_correlation
from sklearn.metrics import pairwise_distances
import matplotlib.animation as animation


In [ ]:
# Define the base paths
srm_data_path = '/Volumes/ARCHIVES/thesis_pipeline/SRM_data'
save_data_dir = '/Volumes/ARCHIVES/thesis_pipeline/data'
save_fig_dir = '/Volumes/ARCHIVES/thesis_pipeline/figures/IS-TR_RSA'

# Ensure that directories for saving are created
os.makedirs(save_data_dir, exist_ok=True)
os.makedirs(save_fig_dir, exist_ok=True)

# Define the regions and number of subjects
roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']
n_subjects = 8  # Adjust according to your data

In [ ]:

# Function to load shared response data for all subjects and regions into a 4D array: (regions x subjects x TRs x features)
def load_feature_time_series_all_rois(roi_names, n_subjects):
    all_feature_time_series = []

    # Loop through each ROI and load the shared response data
    for roi_name in roi_names:
        shared_data_file = os.path.join(srm_data_path, f"{roi_name}_shareddata.pkl")
        
        if os.path.exists(shared_data_file):
            with open(shared_data_file, 'rb') as file:
                shared_data = pickle.load(file)

            # Stack data into 3D array: subjects x TRs x features for the current ROI
            feature_time_series = np.array([shared_data[subj_idx].T for subj_idx in range(n_subjects)])  # TRs x features for each subject
            
            # Append the result to the list for all regions
            all_feature_time_series.append(feature_time_series)
        else:
            raise FileNotFoundError(f"Shared data for {roi_name} not found.")
    
    # Convert the list to a 4D array: regions x subjects x TRs x features
    all_feature_time_series = np.array(all_feature_time_series)
    
    return all_feature_time_series

# Example usage: Load feature time series for all ROIs
roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']
n_subjects = 8  # Adjust according to your data
all_feature_time_series = load_feature_time_series_all_rois(roi_names, n_subjects)

# Display the shape of the loaded feature time series
print(f"Loaded feature time series for all ROIs. Shape: {all_feature_time_series.shape}")



In [ ]:
# Function to compute RSA between subjects for each TR while preserving temporal structure
def compute_rsa_across_trs(feature_time_series):
    n_subjects, n_trs, n_features = feature_time_series.shape
    
    # Initialize an empty matrix to store RSA results: TRs x subjects x subjects
    rsa_matrix = np.zeros((n_trs, n_subjects, n_subjects))
    
    # Loop through each TR and compute the correlation between subjects
    for tr in range(n_trs):
        # Extract feature data across subjects for the current TR
        tr_data = feature_time_series[:, tr, :]  # Shape: subjects x features
        
        # Ensure the data is C-contiguous
        tr_data_contiguous = np.ascontiguousarray(tr_data)  # Ensure data is contiguous
        
        # Compute pairwise correlations between subjects for the current TR
        rsa_matrix[tr, :, :] = compute_correlation(tr_data_contiguous, tr_data_contiguous)
    
    return rsa_matrix

# Function to compute and save RSA matrices for all regions
def compute_rsa_for_all_rois(roi_names, n_subjects):
    # Directory to save the stacked RSA matrices for all ROIs
    save_dir = os.path.join(save_data_dir, 'IS_TR_RSA_ROIS')
    os.makedirs(save_dir, exist_ok=True)
    
    # Initialize a list to store RSA matrices for all regions
    rsa_all_rois = []
    
    # Loop through each ROI and compute RSA
    for roi_name in roi_names:
        print(f"Processing ROI: {roi_name}")
        
        # Load feature time series for the current ROI
        feature_time_series = load_feature_time_series(roi_name, n_subjects)
        
        # Compute the RSA matrix across TRs for the current ROI
        rsa_matrix = compute_rsa_across_trs(feature_time_series)
        
        # Append the result to the list of RSA matrices
        rsa_all_rois.append(rsa_matrix)
    
    # Convert the list to a stacked 4D array: regions x TRs x subjects x subjects
    rsa_all_rois = np.array(rsa_all_rois)
    
    # Save the stacked RSA matrices for all ROIs
    save_path = os.path.join(save_dir, 'IS_TR_RSA_matrices_all_ROIs.npy')
    np.save(save_path, rsa_all_rois)
    
    print(f"Saved IS-TR RSA matrices for all ROIs at {save_path}")

# Example usage: Compute RSA across TRs for all ROIs
compute_rsa_for_all_rois(roi_names, n_subjects)


In [3]:
# Path to the saved .npy file
npy_file = '/Volumes/ARCHIVES/thesis_pipeline/data/IS_TR_RSA_ROIS/IS_TR_RSA_matrices_all_ROIs.npy'

# Load and inspect the array
try:
    loaded_data = np.load(npy_file)
    print(f"Loaded data shape: {loaded_data.shape}")
    print(f"Data type: {loaded_data.dtype}")
    
    # Check dimensions: should be (regions, TRs, subjects, subjects)
    regions, trs, subjects, subjects_dim = loaded_data.shape
    print(f"Number of regions: {regions}")
    print(f"Number of TRs: {trs}")
    print(f"Number of subjects: {subjects}")
    
except FileNotFoundError:
    print(f"File {npy_file} not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Loaded data shape: (6, 1291, 8, 8)
Data type: float64
Number of regions: 6
Number of TRs: 1291
Number of subjects: 8


In [ ]:
# Save the 3D feature time series array
time_series_save_path = os.path.join(save_data_dir, f'SRM_3D_Time_series_{roi_name}.npy')
np.save(time_series_save_path, feature_time_series)
print(f"Saved feature time series for {roi_name} at {time_series_save_path}")

# Function to load and visualize the 3D matrix (subjects x TRs x features)
def visualize_3d_matrix(roi_name):
    time_series_load_path = os.path.join(save_data_dir, f'SRM_3D_Time_series_{roi_name}.npy')

    # Load the 3D matrix
    if os.path.exists(time_series_load_path):
        feature_time_series = np.load(time_series_load_path)
        print(f"Loaded 3D time series matrix for {roi_name}. Shape: {feature_time_series.shape}")
        
        # Visualize the matrix by taking the mean across subjects and features
        mean_across_subjects = np.mean(feature_time_series, axis=0)  # TRs x features
        mean_across_features = np.mean(mean_across_subjects, axis=1)  # Mean across features, resulting in TRs
        
        # Plot the mean time series (across subjects and features)
        plt.figure(figsize=(10, 6))
        plt.plot(mean_across_features)
        plt.title(f'Mean Time Series across Subjects and Features for {roi_name}')
        plt.xlabel('TRs')
        plt.ylabel('Mean Feature Value')
        plt.grid(True)
        
        # Save the figure
        save_path = os.path.join(save_fig_dir, f'{roi_name}_mean_time_series.png')
        plt.savefig(save_path, dpi=300)
        plt.close()
        print(f"Saved mean time series plot for {roi_name} at {save_path}")
        
    else:
        raise FileNotFoundError(f"Saved 3D time series for {roi_name} not found.")

# Visualize the saved 3D matrix for a specific ROI (e.g., 'IFG')
visualize_3d_matrix('IFG')


In [ ]:
# Function to plot the RSA matrix for a specific TR
def plot_rsa_for_tr(rsa_matrix, tr, roi_name):
    plt.figure(figsize=(8, 6))
    sns.heatmap(rsa_matrix[tr, :, :], annot=False, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title(f'RSA Matrix for TR {tr} ({roi_name})')
    plt.xlabel('Subjects')
    plt.ylabel('Subjects')
    
    # Save the figure
    save_path = os.path.join(save_fig_dir, f'{roi_name}_RSA_TR_{tr}.png')
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Saved RSA matrix for TR {tr} at {save_path}")

# Plot RSA matrix for a specific TR (e.g., TR 10)
plot_rsa_for_tr(rsa_matrix, tr=10, roi_name='IFG')


In [ ]:
# Function to create an animation of RSA evolution across TRs with a single color bar
def animate_rsa_across_trs(rsa_matrix, roi_name):
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot the first frame (TR 0) to initialize the heatmap
    cax = sns.heatmap(rsa_matrix[0, :, :], annot=False, cmap='coolwarm', vmin=-1, vmax=1, cbar=True, ax=ax)
    
    # Function to update the heatmap for each TR
    def update(tr):
        ax.clear()
        cax = sns.heatmap(rsa_matrix[tr, :, :], annot=False, cmap='coolwarm', vmin=-1, vmax=1, cbar=False, ax=ax)
        ax.set_title(f'RSA Matrix for TR {tr} ({roi_name})')
        ax.set_xlabel('Subjects')
        ax.set_ylabel('Subjects')
    
    # Create the animation
    ani = animation.FuncAnimation(fig, update, frames=rsa_matrix.shape[0], repeat=False)
    
    # Save the animation as a GIF or MP4
    save_path = os.path.join(save_fig_dir, f'{roi_name}_RSA_across_TRs.gif')
    ani.save(save_path, dpi=80, writer='imagemagick')
    print(f"Saved RSA evolution animation for {roi_name} at {save_path}")


animate_rsa_across_trs(rsa_matrix, 'IFG')



In [ ]:

# Function to create a GIF from the saved RSA matrices (.npy file) with a consistent color scale
def create_rsa_gif_from_npy(roi_name):
    # Load the saved RSA matrices
    npy_file = f'/Volumes/ARCHIVES/thesis_pipeline/data/feature_IS_RSA_{roi_name}/{roi_name}_feature_RSA_matrices.npy'
    rsa_matrices = np.load(npy_file)
    
    # Get the number of features
    n_features, n_subjects, _ = rsa_matrices.shape
    
    # Create a figure for the GIF
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot the first frame (feature 0) to initialize the heatmap and color bar
    sns.heatmap(rsa_matrices[0, :, :], annot=False, cmap='coolwarm', vmin=-1, vmax=1, cbar=True, ax=ax)
    
    # Function to update the heatmap for each feature without adding a new color bar
    def update(feature):
        ax.clear()
        sns.heatmap(rsa_matrices[feature, :, :], annot=False, cmap='coolwarm', vmin=-1, vmax=1, cbar=False, ax=ax)
        ax.set_title(f'RSA Matrix for Feature {feature} ({roi_name})')
        ax.set_xlabel('Subjects')
        ax.set_ylabel('Subjects')
    
    # Create the animation
    ani = animation.FuncAnimation(fig, update, frames=n_features, repeat=False)
    
    # Save the animation as a GIF
    save_dir = f'/Volumes/ARCHIVES/thesis_pipeline/figures'
    os.makedirs(save_dir, exist_ok=True)  # Ensure the figures directory exists
    save_path = os.path.join(save_dir, f'feature_TR_IS_RSA_{roi_name}.gif')
    ani.save(save_path, dpi=80, writer='imagemagick')
    
    print(f"Saved RSA GIF for {roi_name} at {save_path}")

# Example usage for 'IFG' region
roi_name = 'IFG'
create_rsa_gif_from_npy(roi_name)


In [ ]:
# Function to compute RSA between subjects for each feature (node) using BrainIAK's compute_correlation and save the results
def compute_rsa_across_features_and_save(feature_time_series, roi_name):
    n_subjects, n_trs, n_features = feature_time_series.shape
    
    # Initialize an empty list to store RSA results for each feature
    rsa_matrices = []
    
    # Loop through each feature and compute subject-by-subject correlation using compute_correlation
    for feature in range(n_features):
        # Extract data across subjects for the current feature
        feature_data = feature_time_series[:, :, feature]  # Shape: subjects x TRs
        
        # Ensure the data is C-contiguous
        feature_data_contiguous = np.ascontiguousarray(feature_data)  # Shape: subjects x TRs
        
        # Compute pairwise correlations between subjects for this feature using BrainIAK's compute_correlation
        similarity_matrix = compute_correlation(feature_data_contiguous, feature_data_contiguous)
        
        # Store the similarity matrix
        rsa_matrices.append(similarity_matrix)
    
    # Convert list to 3D array: features x subjects x subjects
    rsa_matrices = np.array(rsa_matrices)
    
    # Save the RSA matrices for this ROI in the designated folder
    save_dir = f'/Volumes/ARCHIVES/thesis_pipeline/data/feature_IS_RSA_brainiak{roi_name}'
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
    
    # Save the feature RSA matrices as an npy file
    save_path = os.path.join(save_dir, f'{roi_name}_feature_RSA_matrices_brainiak.npy')
    np.save(save_path, rsa_matrices)
    
    print(f"Saved BrainIAK feature-based RSA matrices for {roi_name} at {save_path}")

# Example usage for 'IFG' region
roi_name = 'IFG'
compute_rsa_across_features_and_save(feature_time_series, roi_name)


In [ ]:

# Function to create a GIF from the saved BrainIAK feature RSA matrices (.npy file)
def create_rsa_gif_from_npy_brainiak(roi_name):
    # Corrected file path for the BrainIAK RSA matrices
    npy_file = f'/Volumes/ARCHIVES/thesis_pipeline/data/feature_IS_RSA_brainiak{roi_name}/{roi_name}_feature_RSA_matrices_brainiak.npy'
    rsa_matrices = np.load(npy_file)
    
    # Get the number of features
    n_features, n_subjects, _ = rsa_matrices.shape
    
    # Create a figure for the GIF
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot the first frame (feature 0) to initialize the heatmap and color bar
    sns.heatmap(rsa_matrices[0, :, :], annot=False, cmap='coolwarm', vmin=-1, vmax=1, cbar=True, ax=ax)
    
    # Function to update the heatmap for each feature without adding a new color bar
    def update(feature):
        ax.clear()
        sns.heatmap(rsa_matrices[feature, :, :], annot=False, cmap='coolwarm', vmin=-1, vmax=1, cbar=False, ax=ax)
        ax.set_title(f'BrainIAK RSA Matrix for Feature {feature} ({roi_name})')
        ax.set_xlabel('Subjects')
        ax.set_ylabel('Subjects')
    
    # Create the animation
    ani = animation.FuncAnimation(fig, update, frames=n_features, repeat=False)
    
    # Save the animation as a GIF
    save_dir = f'/Volumes/ARCHIVES/thesis_pipeline/figures'
    os.makedirs(save_dir, exist_ok=True)  # Ensure the figures directory exists
    save_path = os.path.join(save_dir, f'{roi_name}_feature_RSA_matrices_brainiak.gif')
    ani.save(save_path, dpi=80, writer='imagemagick')
    
    print(f"Saved BrainIAK RSA GIF for {roi_name} at {save_path}")

# Example usage for 'IFG' region
roi_name = 'IFG'
create_rsa_gif_from_npy_brainiak(roi_name)
